In [1]:
from nltk.stem.porter import *
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
stemmer = PorterStemmer()

In [3]:
full_df = pd.read_csv("data/preprocessed_shows.csv")
metadata = pd.read_csv("data/testgenres1.csv")
df = pd.merge(metadata, full_df, on = "show_name")

In [9]:
df

,show_name,show_uri_x,show_description_x,publisher_x,rss_link_x,avg_duration_x,categories,publish_time,show_uri_y,show_description_y,publisher_y,rss_link_y,ep_descriptions,avg_duration_y,ep_transcripts
0,The Grier Podcast,spotify:show:1spDjPLku0UCnir7H2hEG4,A podcast recorded and created by the students...,Bryan Beckel,https://anchor.fm/s/11399a34/podcast/rss,21.389133,['Education'],2022-02-21 12:00:00,spotify:show:1spDjPLku0UCnir7H2hEG4,A podcast recorded and created by the students...,Bryan Beckel,https://anchor.fm/s/11399a34/podcast/rss,"In this episode, four of our students take tim...",21.389133,"hey, thi is cultur shock a broadcast whi for i..."
1,"""ACIM"" A Course In Miracles For The Millennium",spotify:show:5wb7vWvrCtgsY3jMvIAOXW,"""A study group for the transformation of life ...",Rafael Hidalgo,https://anchor.fm/s/25197c4/podcast/rss,87.984350,['Religion & Spirituality'],2019-02-21 19:27:18,spotify:show:5wb7vWvrCtgsY3jMvIAOXW,"""A study group for the transformation of life ...",Rafael Hidalgo,https://anchor.fm/s/25197c4/podcast/rss,Part 1,87.984350,sphere within a firm for each of us that your ...
2,AfroLatina_BeautifitlyBalanced,spotify:show:3zbdWAjWe2sp91t3Ly1j49,Health+SELF Love+Spotless Clean Home! Thats ev...,The AfroLatina_BeautifitlyBalanced,https://anchor.fm/s/9571594/podcast/rss,19.906850,"['Health & Fitness', 'Mental Health']",2022-07-08 03:24:43,spotify:show:3zbdWAjWe2sp91t3Ly1j49,Health+SELF Love+Spotless Clean Home! Thats ev...,The AfroLatina_BeautifitlyBalanced,https://anchor.fm/s/9571594/podcast/rss,Im going of the top 7 secretstraitsof a confid...,19.906850,thank you for tune in to the appl akina beauti...
3,Apostles of Tomorrow,spotify:show:31r4WwUvNQ0CHGU2xobnZP,"A podcast to help fellow Gen Z Apostolics,To f...",Your host Garrett,https://anchor.fm/s/ddebcac/podcast/rss,29.358917,"['Religion & Spirituality', 'Spirituality']",2020-10-29 17:00:00,spotify:show:31r4WwUvNQ0CHGU2xobnZP,"A podcast to help fellow Gen Z Apostolics,To f...",Your host Garrett,https://anchor.fm/s/ddebcac/podcast/rss,"In this we introduce yall to us,And we talk ab...",29.358917,"hey guys, welcom to the first episod of apostl..."
4,Brave Broccoli,spotify:show:7K1LCJSpW2OoDcI9PU9iyV,Just start talking and we can go from there,The brave vegetable,https://anchor.fm/s/122f9614/podcast/rss,37.889942,"['Comedy', 'Comedy Interviews']",2020-01-19 01:02:16,spotify:show:7K1LCJSpW2OoDcI9PU9iyV,Just start talking and we can go from there,The brave vegetable,https://anchor.fm/s/122f9614/podcast/rss,"The boys discuss Iran, pregnancy, the Soviet U...",37.889942,welcom to the brave broccoli the onli podcast ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15873,free your mind: podcast,spotify:show:2DqGCojWyryoDFq0argY9E,Sharing the journey of life & how to set your ...,Ima,https://anchor.fm/s/e94e5e0/podcast/rss,10.529867,"['Education', 'Self-Improvement']",2021-03-02 03:49:00,spotify:show:2DqGCojWyryoDFq0argY9E,Sharing the journey of life & how to set your ...,Ima,https://anchor.fm/s/e94e5e0/podcast/rss,Welcome to my Podcast! Today you are listening...,10.529867,"hey guys, welcom to my podcast. My name is emm..."
15874,The Best Comes from the worst,spotify:show:31aNxmwRKE2zlQbYVglQxe,In order to live your best life you must exper...,James Taylor,https://anchor.fm/s/b2b7450/podcast/rss,16.416900,['Religion & Spirituality'],2019-10-14 17:02:45,spotify:show:31aNxmwRKE2zlQbYVglQxe,In order to live your best life you must exper...,James Taylor,https://anchor.fm/s/b2b7450/podcast/rss,Failed relationships comes from a lack within ...,16.416900,what' go on soul trap is a boy colombian read ...
15875,RED CUP VIBEz,spotify:show:5omDM65kJwYhlJdAiA86fn,Sipping too many adult beverages out of a red ...,BURBERRYLYTE,https://anchor.fm/s/12949c94/podcast/rss,36.678233,"['Comedy', 'Improv']",2020-01-19 10:15:58,spotify:show:5omDM65kJwYhlJdAiA86fn,Sipping too many adult beverages out of a red ...,BURBERRYLYTE,https://anchor.fm/s/12949c94/podc

In [5]:
df = df.replace(r"^ +| +$", r"", regex=True)

In [6]:
duplicates = df["show_name"].duplicated(keep = False)
df = df[~duplicates]

In [ ]:
for i in range(len(df)):
    transcript = df.iat[i, 14]
    words = transcript.split(" ")
    stemmed_words = [stemmer.stem(word) for word in words]
    df.iat[i, 14] = ' '.join(stemmed_words)
    
    if i % 100 == 0:
        print(str(i / len(df) * 100), "% complete")

In [15]:
new_df = df[df['show_name'].notna()]

In [18]:
new_df['show_name'].isnull().values.any()

False

In [20]:
df = new_df

In [22]:
shows = df.set_index('show_name').to_dict('index')

show_name_to_index = {show_name : index for index, show_name in enumerate([show_name for show_name in shows])}
show_index_to_name = {v:k for k,v in show_name_to_index.items()}

show_names = [show_name for show_name in shows]

In [24]:
n_feats = 20000

def build_vectorizer(max_features, stop_words, max_df=0.8, min_df=10, norm='l2'):
    """Returns a TfidfVectorizer object with the above preprocessing properties.
    
    Note: This function may log a deprecation warning. This is normal, and you
    can simply ignore it.
    
    Parameters
    ----------
    max_features : int
        Corresponds to 'max_features' parameter of the sklearn TfidfVectorizer 
        constructer.
    stop_words : str
        Corresponds to 'stop_words' parameter of the sklearn TfidfVectorizer constructer. 
    max_df : float
        Corresponds to 'max_df' parameter of the sklearn TfidfVectorizer constructer. 
    min_df : float
        Corresponds to 'min_df' parameter of the sklearn TfidfVectorizer constructer. 
    norm : str
        Corresponds to 'norm' parameter of the sklearn TfidfVectorizer constructer. 

    Returns
    -------
    TfidfVectorizer
        A TfidfVectorizer object with the given parameters as its preprocessing properties.
    """
    return TfidfVectorizer(max_features = max_features, stop_words = stop_words, max_df = 0.9, min_df = 5, norm = norm)

In [26]:
tfidf_vec = build_vectorizer(n_feats, stop_words = "english")
doc_by_vocab = tfidf_vec.fit_transform([shows[show_name]['ep_transcripts'] for show_name in shows]).toarray()
index_to_vocab = {i:v for i, v in enumerate(tfidf_vec.get_feature_names())}

/Users/william/CS/4300/cs4300-env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [28]:
import pickle
pickle.dump(tfidf_vec, open( "tfidf_stem.p", "wb" ) )

In [37]:
doc_by_vocab.shape

(15842, 20000)

In [57]:
from scipy.sparse.linalg import svds
# do SVD with a very large k (we usually use 100), just for the sake of getting many sorted singular values (aka importances)
docs_compressed, s, words_compressed = svds(doc_by_vocab, k = 40)

In [60]:
pickle.dump(docs_compressed, open( "docs.p", "wb" ) )
pickle.dump(s, open( "s.p", "wb" ) )
pickle.dump(words_compressed, open( "words.p", "wb" ) )

In [61]:
from sklearn.preprocessing import normalize

In [62]:
words_compressed = words_compressed.transpose()
words_compressed_normed = normalize(words_compressed, axis = 1)
docs_compressed_normed = normalize(docs_compressed)

In [59]:
docs_compressed_normed.shape, s.shape, words_compressed.shape

((15842, 40), (40,), (40, 20000))

In [63]:
publisher_to_show_name = {}

for show_name in shows:
    publisher = shows[show_name]["publisher_x"]
    if publisher in publisher_to_show_name:
        publisher_to_show_name[publisher].append(show_name)
    else:
        publisher_to_show_name[publisher] = [show_name]

In [64]:
len(publisher_to_show_name)

15421

In [65]:
category_to_show_name = {}

for show_name in shows:
    category_string = shows[show_name]["categories"]
    category_list = category_string.strip('][').split(', ')
    for category in category_list:
        category = category.strip("'")
        if category in category_to_show_name:
            category_to_show_name[category].append(show_name)
        else:
            category_to_show_name[category] = [show_name]

In [66]:
publisher_tf_idf = {}

for publisher in publisher_to_show_name:
    shows_by_publisher = publisher_to_show_name[publisher]
    
    tf_idf_vec = np.zeros(docs_compressed_normed.shape[1])
    
    for show in shows_by_publisher:
        
        show_idx = show_name_to_index[show]
        show_tf_idf_vec = docs_compressed_normed[show_idx, :]
        tf_idf_vec += show_tf_idf_vec
    
    publisher_tf_idf[publisher] = tf_idf_vec / len(shows_by_publisher)

In [67]:
publisher_tf_idf

{'Bryan Beckel': array([ 0.15071042, -0.3243923 ,  0.16570354, -0.00120968, -0.17585498,
         0.19354327,  0.13300296,  0.08166996, -0.10114791, -0.08840808,
        -0.05334336, -0.03345084,  0.08197516, -0.17016403, -0.35989933,
        -0.10861074,  0.20163288, -0.09356092,  0.30498579,  0.27642451,
         0.08886839, -0.1193224 ,  0.09481925, -0.04798702,  0.09286457,
        -0.13332346,  0.00451334, -0.07390768, -0.15807097, -0.11225411,
        -0.03729163,  0.07268191,  0.00354882, -0.00218652, -0.01483436,
        -0.01425645,  0.26779653,  0.05084804,  0.32444804, -0.21077693]),
 'Rafael Hidalgo': array([ 0.01464139, -0.08257599, -0.1493895 ,  0.30035053,  0.01812892,
        -0.03246967,  0.14789192, -0.01171197,  0.05763405, -0.09313711,
        -0.22930962,  0.10726129,  0.02343809,  0.11638266, -0.04173673,
         0.01342328,  0.25133645, -0.16119264, -0.25841916, -0.28123546,
        -0.04782747, -0.29949506, -0.29444912, -0.06170419,  0.07627802,
        -0.1576

In [68]:
category_tf_idf = {}

for category in category_to_show_name:
    shows_in_category = category_to_show_name[category]
    
    tf_idf_vec = np.zeros(docs_compressed_normed.shape[1])
    
    for show in shows_in_category:
        
        show_idx = show_name_to_index[show]
        show_tf_idf_vec = docs_compressed_normed[show_idx, :]
        tf_idf_vec += show_tf_idf_vec
    
    category_tf_idf[category] = tf_idf_vec / len(shows_in_category)

In [69]:
category_tf_idf

{'After Shows': array([-0.08165934,  0.05897872,  0.01659405,  0.03877304,  0.01234583,
        -0.07934091, -0.00531176, -0.01819114, -0.06755035,  0.05277369,
        -0.01847786,  0.07146363, -0.01223836, -0.01226216,  0.00059599,
        -0.04338539, -0.01546827, -0.00318961, -0.06608881,  0.0413129 ,
        -0.10695686,  0.1127665 ,  0.11308388, -0.02662065, -0.01067566,
         0.06810179,  0.08307592, -0.01238004, -0.03567795,  0.02961998,
        -0.09725645,  0.01120893,  0.03709713,  0.04506323,  0.16978293,
        -0.11343555,  0.03786472,  0.01648066,  0.08797804, -0.20881666]),
 'Alternative Health': array([ 0.03602423,  0.08162677, -0.00134078,  0.02227123, -0.0227192 ,
        -0.06304351, -0.02916333,  0.03019486,  0.04376295, -0.0504997 ,
         0.04228518, -0.10479141,  0.14621534, -0.01378495,  0.02521471,
         0.05685044,  0.01696353,  0.03454232,  0.01529371,  0.07873046,
        -0.05540855, -0.08553049,  0.02167932, -0.02938377, -0.00495437,
        -0.0

In [70]:
pickle.dump(publisher_tf_idf, open( "publish.p", "wb" ) )
pickle.dump(category_tf_idf, open( "category.p", "wb" ) )